In [1]:
!pip install --upgrade pip
!pip install ultralytics==8.3.97
!pip install -U albumentations optuna huggingface_hub
!git clone https://github.com/sathishkumar67/ADIS.git
!mv /kaggle/working/ADIS/* /kaggle/working/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 69.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 46.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 70.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 87.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 84.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 33.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found 

In [2]:
# necessary imports
from __future__ import annotations
import os
import joblib
import optuna
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Define the global variables
REPO_ID = "pt-sk/ADIS" 
FILENAME_IN_REPO = "dataset.zip"
LOCAL_DIR = os.getcwd()
TRAIN_PATH = f"{LOCAL_DIR}/dataset/train"
VAL_PATH = f"{LOCAL_DIR}/dataset/val"
TEST_PATH = f"{LOCAL_DIR}/dataset/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10                                               
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake'] 
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"
MODEL_PATH = "yolo11s.pt"

In [5]:
# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)

# remove dataset.zip
os.remove("/kaggle/working/dataset.zip")

# Get the number of CPU cores
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

dataset.zip:   0%|          | 0.00/9.57G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 9.60G/9.60G [01:01<00:00, 157MB/s] 


Number of CPU cores: 4


In [7]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    print("data yaml file written!.............")

data yaml file written!.............


In [8]:
# Define the objective function
def objective(trial):
    
    # Define callback to report intermediate results
    def on_train_epoch_end(score, epoch):
        trial.report(score, step=epoch)  
        if trial.should_prune():
            raise optuna.TrialPruned()

    callbacks = {
        "on_train_epoch_end" : on_train_epoch_end
    }
    
    # Define hyperparameters using Optuna suggestions
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
    lrf = trial.suggest_float("lrf", 0.1, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.0001, 0.01, log=True)
    warmup_momentum = trial.suggest_float("warmup_momentum", 0.5, 0.9)
    momentum = trial.suggest_float("momentum", 0.8, 0.99)
    
    CONFIG_DICT = {
    "task": "detect",
    "mode": "train",
    "bohb": True,
    "custom_callbacks": callbacks,
    "data": DATA_YAML_FILE,
    "batch": 160,
    "imgsz": 320,
    "save": True,
    "device": 0,
    "workers": num_cores,
    "pretrained": True,
    "optimizer": "AdamW",
    "seed": 42,
    "epochs": 20,
    "warmup_epochs": 3,
    "patience": 4}

    # Train YOLO model
    model = YOLO11Model(MODEL_PATH)
    model.train(**CONFIG_DICT, lr0=lr0, lrf=lrf, momentum=momentum, weight_decay=weight_decay, warmup_momentum=warmup_momentum)
    
    # Return validation mAP as the objective value
    return model.score 

In [ ]:
import joblib
NUM_TRIALS = 1

# load the study
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.TPESampler(), 
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="yolo11_tuning",
                            load_if_exists=True)

# Optimize with a callback to stop after NUM_TRIALS complete trials
study.optimize(
    objective,
    n_trials=NUM_TRIALS)

joblib.dump(study, "/kaggle/working/optuna_study.pkl")

[I 2025-04-18 10:13:29,833] A new study created in memory with name: yolo11_tuning


New https://pypi.org/project/ultralytics/8.3.111 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.97 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data.yaml, epochs=20, time=None, patience=4, batch=160, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

100%|██████████| 755k/755k [00:00<00:00, 17.8MB/s]
E0000 00:00:1744971216.167399      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744971216.225256      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  blocks.Conv                                  [3, 16, 3, 2]                 
  1                  -1  1      4672  blocks.Conv                                  [16, 32, 3, 2]                
  2                  -1  1      6640  blocks.C3k2                                  [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  blocks.Conv                                  [64, 64, 3, 2]                
  4                  -1  1     26080  blocks.C3k2                                  [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  blocks.Conv                                  [128, 128, 3, 2]              
  6                  -1  1     87040  blocks.C3k2                                  [128, 128, 1, True]           
  7                  -1  1    295424  blocks.Con

train: Scanning /kaggle/working/dataset/train... 20000 images, 4 backgrounds, 0 corrupt: 100%|██████████| 20000/20000 [00:16<00:00, 1236.86it/s]


...

       2/20      6.13G      1.277      2.227      1.317        453        320:  15%|█▌        | 19/125 [00:12<01:40,  1.05it/s]libpng warning: iCCP: known incorrect sRGB profile
       2/20      6.13G      1.256      2.044      1.303        473        320: 100%|██████████| 125/125 [01:43<00:00,  1.21it/s]

Epoch 2: AVG Box Loss: 1.2555 | AVG Cls Loss: 2.0443 | AVG DFL Loss: 1.3027



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  40%|████      | 2/5 [00:05<00:07,  2.41s/it]libpng warning: iCCP: known incorrect sRGB profile
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:08<00:00,  1.79s/it]


                   all       1400       1781      0.335      0.365      0.296      0.172
Epoch 2: AVG Val Box Loss: 1.4990 | AVG Val Cls Loss: 2.9895 | AVG Val DFL Loss: 1.5932 | Total Val Loss: 6.0816





      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/20      6.14G      1.249      1.877      1.286        489        320:  10%|█         | 13/125 [00:07<01:20,  1.39it/s]libpng warning: iCCP: known incorrect sRGB profile
       3/20      6.14G      1.247      1.878      1.286        452        320:  13%|█▎        | 16/125 [00:09<01:10,  1.55it/s]